
# Column Transformer with Mixed Types


This example illustrates how to apply different preprocessing and feature
extraction pipelines to different subsets of features, using
`ColumnTransformer`. This is particularly handy for the
case of datasets that contain heterogeneous data types, since we may want to
scale the numeric features and one-hot encode the categorical ones.

In this example, the numeric data is standard-scaled after mean-imputation,
while the categorical data is one-hot encoded after imputing missing values
with a new category (``'missing'``).

In addition, we show two different ways to dispatch the columns to the
particular pre-processor: by column names and by column data types.

Finally, the preprocessing pipeline is integrated in a full prediction pipeline
using `Pipeline`, together with a simple classification
model.


In [1]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


#### Step 1: Read the data set `Titanic.csv`
Print the column names. Drop 'boat','body','cabin','home.dest','name','ticket','sibsp','parch'

In [2]:
df = pd.read_csv('Titanic.csv')
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,?,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,?,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30,1,2,113781,151.55,C22 C26,S,?,135,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"


In [ ]:
df.drop(['boat', 'body', 'cabin', 'home.dest', 'name', 'ticket', 'sibsp', 'parch'], inplace=True, axis=1)

#### Step 2: Calculate duplicates
- Report the size of the dataset
- Report if there are any duplicates using `duplicated()` method of pandas dataframe
- List all duplicate rows
- Drop all duplicate rows using `dop_duplicates` method of pandas dataframe
- Report the new shape of the dataset

In [4]:
# # Report the size of the dataset
# original_shape = df.shape
# print("Original dataset shape:", original_shape)

# # Report if there are any duplicates
# duplicates_exist = df.duplicated().any()
# print("Are there duplicates in the dataset?", duplicates_exist)

# # List all duplicate rows
# duplicate_rows = df[df.duplicated(keep=False)]
# print("Duplicate rows:")
# print(duplicate_rows)

# # Drop all duplicate rows
# df.drop_duplicates(inplace=True)

# # Report the new shape of the dataset
# new_shape = df.shape
# print("New dataset shape after dropping duplicates:", new_shape)


Original dataset shape: (1309, 6)
Are there duplicates in the dataset? True
Duplicate rows:
      pclass  survived     sex age     fare embarked
12         1         1  female  24     69.3        C
34         1         0    male  42    26.55        S
70         1         0    male   ?        0        S
80         1         0    male   ?    26.55        S
118        1         0    male   ?    26.55        S
...      ...       ...     ...  ..      ...      ...
1296       3         0    male  27   8.6625        S
1297       3         0    male   ?     7.25        S
1302       3         0    male   ?    7.225        C
1303       3         0    male   ?  14.4583        C
1305       3         0  female   ?  14.4542        C

[318 rows x 6 columns]
New dataset shape after dropping duplicates: (1094, 6)


In [7]:
df.head()

,pclass,survived,sex,age,fare,embarked
0,1,1,female,29,211.3375,S
1,1,1,male,0.9167,151.55,S
2,1,0,female,2,151.55,S
3,1,0,male,30,151.55,S
4,1,0,female,25,151.55,S


In [9]:
df.shape

(1309, 6)

In [12]:
df[df.duplicated()]

,pclass,survived,sex,age,fare,embarked
118,1,0,male,?,26.55,S
125,1,0,male,?,0,S
179,1,0,male,?,26.55,S
185,1,0,male,42,26.55,S
223,1,0,male,?,0,S
...,...,...,...,...,...,...
1296,3,0,male,27,8.6625,S
1297,3,0,male,?,7.25,S
1302,3,0,male,?,7.225,C
1303,3,0,male,?,14.4583,C


In [17]:
df.drop_duplicates(inplace=True)
# df.drop_duplicates(subset=['pclass'])

In [18]:
df.shape

(1094, 6)

#### Step 3: Explore the dataset for missing values
- Assign `survived' as target variable `y` and the rest as `X`
- Print info about `X` to see if there are any null values and the type of the features
- Check if all columns have only alpha numeric characters. To do that you need to use `isalnum()` method of Python which works on strings. Before using this method. we need to change all features into string using `astype('str')`
- Replace all `?` characters with `np.nan`
- Now, turn 'age','fare' into numeric again using `apply(pd.to_numeric) `
- Print the first five samples of `X`
- Summarize the number of unique values in each column using `nunique()`

In [5]:
# # Assign 'survived' as target variable 'y' and the rest as 'X'
# y = df['survived']
# X = df.drop(columns=['survived'])

# # Print info about X to check for null values and feature types
# print("Info about X:")
# print(X.info())

# # Check if all columns have only alphanumeric characters
# all_alphanumeric = X.astype(str).apply(lambda col: col.str.isalnum().all())
# print("\nDo all columns have only alphanumeric characters?")
# print(all_alphanumeric)

# # Replace all '?' characters with np.nan
# X.replace('?', np.nan, inplace=True)

# # Turn 'age' and 'fare' back into numeric
# X['age'] = X['age'].apply(pd.to_numeric, errors='coerce')
# X['fare'] = X['fare'].apply(pd.to_numeric, errors='coerce')

# # Print the first five samples of X
# print("\nFirst five samples of X:")
# print(X.head())

# # Summarize the number of unique values in each column
# unique_counts = X.nunique()
# print("\nNumber of unique values in each column:")
# print(unique_counts)


Info about X:
<class 'pandas.core.frame.DataFrame'>
Index: 1094 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pclass    1094 non-null   int64 
 1   sex       1094 non-null   object
 2   age       1094 non-null   object
 3   fare      1094 non-null   object
 4   embarked  1094 non-null   object
dtypes: int64(1), object(4)
memory usage: 51.3+ KB
None

Do all columns have only alphanumeric characters?
pclass       True
sex          True
age         False
fare        False
embarked    False
dtype: bool

First five samples of X:
   pclass     sex      age      fare embarked
0       1  female  29.0000  211.3375        S
1       1    male   0.9167  151.5500        S
2       1  female   2.0000  151.5500        S
3       1    male  30.0000  151.5500        S
4       1  female  25.0000  151.5500        S

Number of unique values in each column:
pclass        3
sex           2
age          98
fare        281
emb

In [19]:
y = df['survived']
X = df.drop('survived', axis=1)

In [20]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1094 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pclass    1094 non-null   int64 
 1   sex       1094 non-null   object
 2   age       1094 non-null   object
 3   fare      1094 non-null   object
 4   embarked  1094 non-null   object
dtypes: int64(1), object(4)
memory usage: 51.3+ KB


In [23]:
df.describe()

,pclass,survived
count,1094.000000,1094.000000
mean,2.211152,0.423218
std,0.860979,0.494295
min,1.000000,0.000000
25%,1.000000,0.000000
50%,2.000000,0.000000
75%,3.000000,1.000000
max,3.000000,1.000000


In [28]:
a = {'pclass': 'float'}
X = X.astype(a)

In [27]:
X.dtypes

pclass      object
sex         object
age         object
fare        object
embarked    object
dtype: object

In [29]:
cols = X.columns
X[cols] = X[cols].astype('str')

In [30]:
X.dtypes

pclass      object
sex         object
age         object
fare        object
embarked    object
dtype: object

In [32]:
X.isnull()

,pclass,sex,age,fare,embarked
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
1301,False,False,False,False,False
1304,False,False,False,False,False
1306,False,False,False,False,False
1307,False,False,False,False,False


In [33]:
X.isnull().sum(axis=0)

pclass      0
sex         0
age         0
fare        0
embarked    0
dtype: int64

In [43]:
X[X.columns[3]]

0       211.3375
1         151.55
2         151.55
3         151.55
4         151.55
          ...   
1301       7.225
1304     14.4542
1306       7.225
1307       7.225
1308       7.875
Name: fare, Length: 1094, dtype: object

In [ ]:
# ~(X.iloc[:,3].str.isalnum())    # non-alphaNumeric character

X[~(X.iloc[:,3].str.isalnum())]     # non-alphanumeric charactered fare rows

# X[~(X.iloc[:,3].str.isalnum())]['fare'].apply(checkfloat)

In [57]:
X = X.replace({'?':np.nan})

In [60]:
X.dtypes

pclass      object
sex         object
age         object
fare        object
embarked    object
dtype: object

In [63]:
b = {'age': 'float', 'fare': 'float'}
X= X.astype(b)

In [68]:
X.dtypes

pclass       object
sex          object
age         float64
fare        float64
embarked     object
dtype: object

In [65]:
X.isnull().sum(axis=0)

pclass        0
sex           0
age         128
fare          1
embarked      2
dtype: int64

In [66]:
X['embarked'].unique()

array(['S', 'C', nan, 'Q'], dtype=object)

In [67]:
X['embarked'] = X['embarked'].astype('str')

In [69]:
X['embarked'].unique()

array(['S', 'C', 'nan', 'Q'], dtype=object)

#### Step 4: Use `ColumnTransformer` by selecting column by names

##############################################################################
 
 We will train our classifier with the following features:

 Numeric Features:

 * ``age``: float;
 * ``fare``: float.

 Categorical Features:

 * ``embarked``: categories encoded as strings ``{'C', 'S', 'Q'}``;
 * ``sex``: categories encoded as strings ``{'female', 'male'}``;
 * ``pclass``: ordinal integers ``{1, 2, 3}``.

 Create the preprocessing pipelines for both numeric and categorical data.
 - For numeric features, create a pipeline with using imputer with median strategy and standard scaler
 - For categorical features, use imputer with most frequent and onehot encoder
 
 Use columntransformer to transform the features
 
 Append the preprocessor to the KNeighborsClassifier


In [6]:
# # Define numeric and categorical features
# numeric_features = ['age', 'fare']
# categorical_features = ['embarked', 'sex', 'pclass']

# # Create preprocessing pipelines
# numeric_pipeline = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())
# ])

# categorical_pipeline = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# # Create ColumnTransformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_pipeline, numeric_features),
#         ('cat', categorical_pipeline, categorical_features)
#     ]
# )

# # Create the complete pipeline with KNeighborsClassifier
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('classifier', KNeighborsClassifier(n_neighbors=5))
# ])

# # Optional: Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Fit the pipeline on the training data
# pipeline.fit(X_train, y_train)

# # Evaluate the model
# score = pipeline.score(X_test, y_test)
# print("Test score:", score)


Test score: 0.7990867579908676


In [71]:
numeric_features = ['age, fare']
numeric_transformer = Pipeline(steps=[('n_imputer', SimpleImputer(strategy='mean')),
                                      ('scaler', StandardScaler())])

In [72]:
numeric_transformer

Pipeline(steps=[('n_imputer', SimpleImputer()), ('scaler', StandardScaler())])

In [73]:
categorical_features = ['embarked', 'gender', 'pclass']
categorical_transformer = Pipeline(steps=[('c_imputer', SimpleImputer(strategy='most_frequent', 
                                                                      missing_values='nan')),
                                                                      ('ohe', OneHotEncoder(handle_unknown='ignore'))])

In [74]:
categorical_transformer

Pipeline(steps=[('c_imputer',
                 SimpleImputer(missing_values='nan', strategy='most_frequent')),
                ('ohe', OneHotEncoder(handle_unknown='ignore'))])

In [75]:
Preprocessor = ColumnTransformer(transformers = [('num',numeric_transformer, numeric_features),
                                  ('cat', categorical_transformer, categorical_features)])

In [76]:
Preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('n_imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['age, fare']),
                                ('cat',
                                 Pipeline(steps=[('c_imputer',
                                                  SimpleImputer(missing_values='nan',
                                                                strategy='most_frequent')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['embarked', 'gender', 'pclass'])])

In [77]:
classifier = Pipeline(steps=[('prep', Preprocessor),
                ('knn', KNeighborsClassifier())])

In [78]:
classifier

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('n_imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age, fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('c_imputer',
                                                                   SimpleImputer(missing_values='nan',
                                                                                 strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['embarked', 'gender',
                                                   'pclass'])])),
                ('knn', KNeighborsClassifier())])

Using the prediction pipeline in a grid search

##############################################################################

 Grid search can also be performed on the different preprocessing steps
 defined in the `ColumnTransformer` object, together with the classifier's
 hyperparameters as part of the `Pipeline`.
 We will search for both the imputer strategy of the numeric preprocessing
 and the number of neighbor parameter of the kneighbors classifier using
 `GridSearchCV`.



In [7]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],  # Imputer strategies for numeric features
    'classifier__n_neighbors': [3, 5, 7, 9, 11, 13]  # Different values for n_neighbors
}

# Create a GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=1)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

# Evaluate on the test set
test_score = grid_search.score(X_test, y_test)
print("Test score of the best model: ", test_score)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best parameters found:  {'classifier__n_neighbors': 5, 'preprocessor__num__imputer__strategy': 'median'}
Best cross-validation score:  0.7645714285714286
Test score of the best model:  0.7990867579908676


In [2]:
# import numpy as np

In [ ]:
# parameters = {'knn__n_neighbours': np.arange(1, 30, 2),
#               'prep__num_n_imputer__strategy': ['mean', 'median']}

# mygrid = GridSearchCV(classifier, param_grid=parameters, cv=5)

# mygrid.fit(X_train, y_train)

: 

In [ ]:
# mygrid.best_params_